In [1]:
import torch
import torchvision
print(f'Torch version: {torch.__version__}')

Torch version: 0.4.0


# Check h5 file format

In [2]:
import sys
sys.path.append('../visdial-challenge-starter-pytorch/')
sys.path.append('../visdial-challenge-starter-pytorch/visdialch/')

In [3]:
from data.readers import ImageFeaturesHdfReader

In [4]:
val_reader = ImageFeaturesHdfReader(
    '../visdial-challenge-starter-pytorch/data/features_faster_rcnn_x101_val.h5')
test_reader = ImageFeaturesHdfReader(
    '../visdial-challenge-starter-pytorch/data/features_faster_rcnn_x101_test.h5')
train_reader = ImageFeaturesHdfReader(
    '../visdial-challenge-starter-pytorch/data/features_faster_rcnn_x101_train.h5')

In [5]:
print(f'Train has {len(train_reader)} samples')
print(f'Valid has {len(val_reader)} samples')
print(f'Test has {len(test_reader)} samples')

Train has 123287 samples
Valid has 2064 samples
Test has 8000 samples


In [6]:
len(set([id for reader in [train_reader, val_reader, test_reader] for id in reader.image_id_list ]))

133351

# Load images

In [7]:
test_reader.keys()[0]

481561.0

In [8]:
import os
import re
from PIL import Image
len(os.listdir('../visdial-challenge-starter-pytorch/data/images/val2014/'))
test_images = os.listdir('../visdial-challenge-starter-pytorch/data/images/test2014/')
test_images_ids = [float(f[14:-4]) for f in test_images]

In [13]:
splits = (
        'train2014',
        'test2014',
        'val2014'
        'VisualDialog_test2018',
        'VisualDialog_val2018',
)

In [9]:
available_image_ids = {
    float(re.findall(r'([\d]+)\.jpg', f)[0])
    :
    os.path.join(f'../visdial-challenge-starter-pytorch/data/images/{split}/{f}')
    for split in (
        'train2014',
        'test2014',
        'val2014'
        #'VisualDialog_test2018',
        #'VisualDialog_val2018',
    )
    for f in os.listdir(f'../visdial-challenge-starter-pytorch/data/images/{split}/')
    }
len(available_image_ids)

164062

In [10]:
img = Image.open(available_image_ids[list(available_image_ids.keys())[40]])

In [11]:
image_ids_with_features = [
    key
    for reader in [
        train_reader, 
        #test_reader,
        #val_reader,
    ]
    for key in reader.keys()
]
len(image_ids_with_features)

123287

# Load Model

In [84]:
checkpoint_path = '../visdial-challenge-starter-pytorch/data/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar'

In [85]:
model = torch.load(checkpoint_path, map_location='cpu')

/usr/cs/public/project_vis/visdialciss/venv/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/cs/public/project_vis/visdialciss/venv/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.DecoderWithAttention' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/cs/public/project_vis/visdialciss/venv/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.Attention' has changed. you can retrieve the original sou

In [88]:
img_encoder = model['encoder']

In [78]:
device = torch.device('cpu')

In [79]:
import numpy as np

In [80]:
transform = transforms.Compose([
    transforms.Resize((300,300)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [91]:
def get_image_features(img_id):
    img_path = available_image_ids.get(img_id)
    img = Image.open(img_path)
    
    return transform(img).to(device)

In [95]:
img_encoder(get_image_features(9.0).unsqueeze(0)).shape

torch.Size([1, 14, 14, 2048])

In [50]:
tensor = get_image_features(9.0)

In [51]:
c = tensor.float() / 255

In [55]:
c.shape

torch.Size([300, 300, 3])

In [57]:
from torchvision import transforms

In [58]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [15]:
import numpy as np

In [16]:
img_tensor = torch.from_numpy(np.array(img))

In [90]:
%%time
res = img_encoder.resnet(torch.randn(1, 3, 300, 300))
print(res.shape)

torch.Size([1, 2048, 10, 10])
CPU times: user 1.78 s, sys: 31.3 ms, total: 1.81 s
Wall time: 455 ms


In [233]:
res

tensor([[[[ 0.6807,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0086],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0298,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0632],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 2.7850,  0.0405,  0.7342,  ...,  0.2540,  0.0000,  1.8835],
          [ 0.0000,  0.0000,  

In [240]:
torch.cuda.empty_cache()

In [9]:
torch.nn.Conv2d(1, 1, (2, 2)).padding_mode

'zeros'

In [14]:
model['encoder'].resnet[0].

AttributeError: 'Conv2d' object has no attribute 'padding_mode'